# Hands-on introduction to ML training
In this notebook we will take a step back from training models and learn how to prepare data for model training.

In this lesson, we will also learn about a technique called one-hot encoding.

### Step 1: Load and explore data
The first step is figuring out the data source. In this case we will use a pre-existing dataset. We will:
1. Create a folder 'data'
2. Download the file from public github repo using python package "requests" and save the grades_raw.csv file in the data folder.

In [1]:
%config IPCompleter.greedy=True #Helps with auto-complete

import os

try:
    os.mkdir('data')
except OSError as error:
    print(error)

import requests, csv

url = ''
r = requests.get(url)
with open('data/grades_raw.csv', 'w') as f:
  writer = csv.writer(f)
  for line in r.iter_lines():
    writer.writerow(line.decode('utf-8').split(','))

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/grades_raw.csv')

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

### This dataset does not have any missing values

In [4]:
print(data[:5])

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]


[Kaggle Dataset](https://www.kaggle.com/datasets/uciml/student-alcohol-consumption)
The data were obtained in a survey of students math and portuguese language courses in secondary school. It contains a lot of interesting social, gender and study information about students. You can use it for some EDA (Exploratory Data Analysis) or try to predict students final grade.

In [5]:
#Removing G1 and G2 because the final grade (G3) is what we want to predict.
data.drop('G1', axis=1, inplace=True)
data.drop('G2', axis=1, inplace=True)

#Removing school as we would like this model to not be specific to only a small number of schools in the dataset
data.drop('school', axis=1, inplace=True)
data[:2]

,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,F,18,U,GT3,A,4,4,at_home,teacher,course,...,no,no,4,3,4,1,1,3,6,6
1,F,17,U,GT3,T,1,1,at_home,other,course,...,yes,no,5,3,3,1,1,3,4,6


### Changing non-numerical boolean values to 0/1
For now we will treat this as a regression problem and prepare the data to be used for LinearRegression model. As LinearRegression model is a mathematical equation, we need to convert all non-numerical features to numerical values. Features with only two possible values can be simply turned into boolean features (0, 1)

In [10]:
for column in data.columns:
    print(f'{column}: {data[column].nunique()}')

sex: 2
age: 8
address: 2
famsize: 2
Pstatus: 2
Medu: 5
Fedu: 5
Mjob: 5
Fjob: 5
reason: 4
guardian: 3
traveltime: 4
studytime: 4
failures: 4
schoolsup: 2
famsup: 2
paid: 2
activities: 2
nursery: 2
higher: 2
internet: 2
romantic: 2
famrel: 5
freetime: 5
goout: 5
Dalc: 5
Walc: 5
health: 5
absences: 34
G3: 18


### Non-numeric columns with 2 distinct values = Boolean Encoding

### Non-numeric columns with 3+ distinct values = One-hot Encoding

In [11]:
data['sex'].replace({'F':0, 'M':1}, inplace=True)

In [12]:
data['address'].replace({'R':0, 'U':1}, inplace=True)

In [13]:
data['famsize'].replace({'LE3':0, 'GT3':1}, inplace=True)

In [14]:
data['Pstatus'].replace({'A':0, 'T':1}, inplace=True)

### One-hot encoding data with only a few different possible values

For non-numerical features that have more than 2 distinct values, we will use One-hot encoding technique to convert them into a series of booleans. For example, a students school can be either elementary, middle or high.

Originally, the data might look like:

|StudentID|School|
|---|---|
|0|Elementary|
|1|Middle|
|2|High|

After one-hot encoding, it might look like:

|StudentID|Elementary|Middle|High|
|---|---|---|---|
|0|1|0|0|
|1|0|1|0
|2|0|0|1|

0 = Student is not in that school <br>
1 =Student is in that school

In [15]:
# Mother Job is a categorical feature with 5 distinct values. We will use get_dummies to One-Hot encode this feature resulting in "5" boolean features
mjob = pd.get_dummies(data['Mjob'])
print(mjob[:10])

   at_home  health  other  services  teacher
0     True   False  False     False    False
1     True   False  False     False    False
2     True   False  False     False    False
3    False    True  False     False    False
4    False   False   True     False    False
5    False   False  False      True    False
6    False   False   True     False    False
7    False   False   True     False    False
8    False   False  False      True    False
9    False   False   True     False    False


In [16]:
#Renaming column names for clarity as the same job can also be for "father"
mjob.columns = ['m_at_home', 'm_health', 'm_other', 'm_services', 'm_teacher']

In [17]:
#Removing the original Mjob categorical column and adding the newly-created 5 boolean columns

data.drop(['Mjob'], axis=1, inplace=True)
data = pd.concat([data, mjob], axis=1)
print(data[:2])

   sex  age  address  famsize  Pstatus  Medu  Fedu     Fjob  reason guardian  \
0    0   18        1        1        0     4     4  teacher  course   mother   
1    0   17        1        1        1     1     1    other  course   father   

   ...  Dalc  Walc  health absences G3 m_at_home m_health m_other m_services  \
0  ...     1     1       3        6  6      True    False   False      False   
1  ...     1     1       3        4  6      True    False   False      False   

  m_teacher  
0     False  
1     False  

[2 rows x 34 columns]


In [18]:
#Repeat the same steps for Father Job column

fjob = pd.get_dummies(data['Fjob'])
print(fjob[:2])

   at_home  health  other  services  teacher
0    False   False  False     False     True
1    False   False   True     False    False


In [19]:
fjob.columns = ['f_at_home', 'f_health', 'f_other', 'f_services', 'f_teacher']

In [20]:
data.drop(['Fjob'], axis=1, inplace=True)
data = pd.concat([data, fjob], axis=1)
print(data[:2])

   sex  age  address  famsize  Pstatus  Medu  Fedu  reason guardian  \
0    0   18        1        1        0     4     4  course   mother   
1    0   17        1        1        1     1     1  course   father   

   traveltime  ...  m_at_home  m_health m_other m_services m_teacher  \
0           2  ...       True     False   False      False     False   
1           1  ...       True     False   False      False     False   

  f_at_home f_health f_other f_services f_teacher  
0     False    False   False      False      True  
1     False    False    True      False     False  

[2 rows x 38 columns]


In [21]:
reason = pd.get_dummies(data['reason'])
print(reason[:2])

   course   home  other  reputation
0    True  False  False       False
1    True  False  False       False


In [22]:
data.drop(['reason'], axis=1, inplace=True)
data = pd.concat([data, reason], axis=1)
print(data[:2])

   sex  age  address  famsize  Pstatus  Medu  Fedu guardian  traveltime  \
0    0   18        1        1        0     4     4   mother           2   
1    0   17        1        1        1     1     1   father           1   

   studytime  ...  m_teacher f_at_home f_health f_other f_services f_teacher  \
0          2  ...      False     False    False   False      False      True   
1          2  ...      False     False    False    True      False     False   

  course   home  other  reputation  
0   True  False  False       False  
1   True  False  False       False  

[2 rows x 41 columns]


In [23]:
guardian = pd.get_dummies(data['guardian'])
print(guardian[:2])

   father  mother  other
0   False    True  False
1    True   False  False


In [24]:
data.drop(['guardian'], axis=1, inplace=True)
data = pd.concat([data, guardian], axis=1)
print(data[:2])

   sex  age  address  famsize  Pstatus  Medu  Fedu  traveltime  studytime  \
0    0   18        1        1        0     4     4           2          2   
1    0   17        1        1        1     1     1           1          2   

   failures  ... f_other f_services f_teacher course   home  other reputation  \
0         0  ...   False      False      True   True  False  False      False   
1         0  ...    True      False     False   True  False  False      False   

  father  mother  other  
0  False    True  False  
1   True   False  False  

[2 rows x 43 columns]


### As seen above, the value is boolean, True or False. We will convert such values to 0/1

In [25]:
# Replace all "no" / "False" to 0 and "yes" / "True" to 1 across the entire dataset
data.replace({'no':0, 'yes':1}, inplace=True)
data.replace({'False':0, 'True':1}, inplace=True)
data.replace({False:0, True:1}, inplace=True)
print(data[:2])

   sex  age  address  famsize  Pstatus  Medu  Fedu  traveltime  studytime  \
0    0   18        1        1        0     4     4           2          2   
1    0   17        1        1        1     1     1           1          2   

   failures  ...  f_other  f_services  f_teacher  course  home  other  \
0         0  ...        0           0          1       1     0      0   
1         0  ...        1           0          0       1     0      0   

   reputation  father  mother  other  
0           0       0       1      0  
1           0       1       0      0  

[2 rows x 43 columns]


In [26]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   sex         395 non-null    int64
 1   age         395 non-null    int64
 2   address     395 non-null    int64
 3   famsize     395 non-null    int64
 4   Pstatus     395 non-null    int64
 5   Medu        395 non-null    int64
 6   Fedu        395 non-null    int64
 7   traveltime  395 non-null    int64
 8   studytime   395 non-null    int64
 9   failures    395 non-null    int64
 10  schoolsup   395 non-null    int64
 11  famsup      395 non-null    int64
 12  paid        395 non-null    int64
 13  activities  395 non-null    int64
 14  nursery     395 non-null    int64
 15  higher      395 non-null    int64
 16  internet    395 non-null    int64
 17  romantic    395 non-null    int64
 18  famrel      395 non-null    int64
 19  freetime    395 non-null    int64
 20  goout       395 non-null    int6

### Now that all columns are numerical and non-null, they will be saved in another file called <u> grades_cleaned.csv</u>

In [27]:
#Saving to csv

data.to_csv('data/grades_clean.csv', index=False)